In [1]:
import json
import os
import pandas as pd
import re

In [2]:
# get the list of files in the outputs directory
files = os.listdir('outputs')

files

['summary_20250316_015501.txt',
 'how_to_apply_20250316_111228.md',
 'evaluated_applications_20250314_201528.json',
 'evaluated_applications_20250315_172239.json',
 'evaluated_applications_20250316_133722.json',
 'application_pages_20250315_173633.json',
 'summary_20250315_173633.txt',
 'summary_20250316_133722.txt',
 'summary_20250316_014123.txt',
 'evaluated_applications_20250316_110959.json',
 'how_to_apply_20250316_133723.csv',
 'evaluated_applications_20250315_170448.json',
 'application_pages_20250316_133722.json',
 'application_pages_20250315_163706.json',
 'how_to_apply_20250316_111228.csv',
 'evaluated_applications_20250316_014123.json',
 'summary_20250316_110959.txt',
 'application_pages_20250315_164305.json',
 'evaluated_applications_20250315_171320.json',
 'summary_20250315_163706.txt',
 'summary_20250316_111228.txt',
 'application_pages_20250315_172239.json',
 'evaluated_applications_20250315_171703.json',
 'evaluated_applications_20250315_165527.json',
 'evaluated_applica

In [3]:
# filter the files that match the regex pattern
pattern = re.compile(r'application_pages_.*\.json')
filtered_files = [file for file in files if pattern.match(file)]

filtered_files

['application_pages_20250315_173633.json',
 'application_pages_20250316_133722.json',
 'application_pages_20250315_163706.json',
 'application_pages_20250315_164305.json',
 'application_pages_20250315_172239.json',
 'application_pages_20250316_015501.json',
 'application_pages_20250316_110959.json',
 'application_pages_20250315_171703.json',
 'application_pages_20250316_104021.json',
 'application_pages_20250315_171320.json',
 'application_pages_20250316_111228.json',
 'application_pages_20250315_165527.json',
 'application_pages_20250314_201528.json',
 'application_pages_20250315_172447.json',
 'application_pages_20250316_012631.json',
 'application_pages_20250316_014123.json',
 'application_pages_20250316_105307.json',
 'application_pages_20250316_113738.json',
 'application_pages_20250315_170448.json']

In [4]:
# create an empty DataFrame
df = pd.DataFrame(columns=['filename', "university", "url"])

df.head()

,filename,university,url


In [5]:
for file in filtered_files:
    with open(os.path.join('outputs', file), 'r') as f:
        data = json.load(f)
        for entry in data:
            df = pd.concat([df, pd.DataFrame([{
                'filename': file,
                'university': entry.get('university', ''),
                'url': entry.get('url', '')
            }])], ignore_index=True)

df.head()

,filename,university,url
0,application_pages_20250315_173633.json,University of Cambridge,https://www.undergraduate.study.cam.ac.uk/find...
1,application_pages_20250315_173633.json,University of Cambridge,https://www.undergraduate.study.cam.ac.uk/inte...
2,application_pages_20250315_173633.json,University of Cambridge,https://www.undergraduate.study.cam.ac.uk/coll...
3,application_pages_20250315_173633.json,University of Cambridge,https://www.undergraduate.study.cam.ac.uk?ucam...
4,application_pages_20250315_173633.json,University of Cambridge,https://www.undergraduate.study.cam.ac.uk/cour...


In [6]:
# write the DataFrame to a csv file
df.to_csv('results.csv', index=False)

More Intense Section

In [7]:
# get the list of files in the outputs directory
files = os.listdir('outputs')

files

['summary_20250316_015501.txt',
 'how_to_apply_20250316_111228.md',
 'evaluated_applications_20250314_201528.json',
 'evaluated_applications_20250315_172239.json',
 'evaluated_applications_20250316_133722.json',
 'application_pages_20250315_173633.json',
 'summary_20250315_173633.txt',
 'summary_20250316_133722.txt',
 'summary_20250316_014123.txt',
 'evaluated_applications_20250316_110959.json',
 'how_to_apply_20250316_133723.csv',
 'evaluated_applications_20250315_170448.json',
 'application_pages_20250316_133722.json',
 'application_pages_20250315_163706.json',
 'how_to_apply_20250316_111228.csv',
 'evaluated_applications_20250316_014123.json',
 'summary_20250316_110959.txt',
 'application_pages_20250315_164305.json',
 'evaluated_applications_20250315_171320.json',
 'summary_20250315_163706.txt',
 'summary_20250316_111228.txt',
 'application_pages_20250315_172239.json',
 'evaluated_applications_20250315_171703.json',
 'evaluated_applications_20250315_165527.json',
 'evaluated_applica

In [2]:
df = pd.DataFrame(columns=['filename', "university", "url"])

In [3]:
current_run = "run_20250317034037_82870f5a"
files = os.listdir(os.path.join('outputs', "checkpoints", current_run))

files

['evaluated_20250317_034122.json',
 'crawler_state.json',
 'how_to_apply_20250317_034323.md',
 'pending_20250317_034323.json',
 'pending_20250317_034137.json',
 'evaluated_20250317_034137.json',
 'evaluated_all.json',
 'how_to_apply_20250317_034122.md',
 'pending_20250317_034231.json',
 'pending_20250317_034122.json',
 'how_to_apply_20250317_034137.md',
 'evaluated_20250317_034231.json',
 'how_to_apply_20250317_034231.md',
 'run_info.json',
 'evaluated_20250317_034323.json']

In [4]:
# filter the files that match the regex pattern
pattern = re.compile(r'evaluated_*.*\.json')
filtered_files = [file for file in files if pattern.match(file)]

filtered_files

['evaluated_20250317_034122.json',
 'evaluated_20250317_034137.json',
 'evaluated_all.json',
 'evaluated_20250317_034231.json',
 'evaluated_20250317_034323.json']

In [6]:
for file in filtered_files:
    with open(os.path.join('outputs', "checkpoints", current_run, file), 'r') as f:
        data = json.load(f)
        
        # print(data)
        print("Processing file: ", file)
        
        for entry in data:
            df = pd.concat([df, pd.DataFrame([{
                'filename': file,
                'university': entry.get('university', ''),
                'url': entry.get('url', '')
            }])], ignore_index=True)

df.head()

Processing file:  evaluated_20250317_034122.json
Processing file:  evaluated_20250317_034137.json
Processing file:  evaluated_all.json
Processing file:  evaluated_20250317_034231.json
Processing file:  evaluated_20250317_034323.json


,filename,university,url
0,evaluated_20250317_034122.json,The University of Edinburgh,https://www.ed.ac.uk/
1,evaluated_20250317_034122.json,The University of Edinburgh,https://study.ed.ac.uk/programmes/undergraduat...
2,evaluated_20250317_034122.json,The University of Edinburgh,https://study.ed.ac.uk/undergraduate/access-ed...
3,evaluated_20250317_034122.json,The University of Sydney,https://www.sydney.edu.au/study/study-options/...
4,evaluated_20250317_034122.json,Tsinghua University,https://www.tsinghua.edu.cn/en/Admissions.htm
